In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Tobigs/컨퍼런스_와인추천/

import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from time import perf_counter
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score
import itertools
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_json('Data 최종본/train_all_meta_v2.json')
test_df = pd.read_json('Data 최종본/test_all_meta_v2.json')

X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','food','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count']]
y_train = train_df['like']
X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','food','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count']]
y_test = test_df['like']

In [ ]:
ALL_FIELDS = X_train.columns
CAT_FIELDS = ['food']
CONT_FIELDS = list(set(ALL_FIELDS).difference(CAT_FIELDS))

# Parameters
BATCH_SIZE = 256
EMBEDDING_SIZE = 1
EPOCHS = 20

class에 알맞게 전처리

In [ ]:
# X_train field에 맞춰 modify
field_dict = dict()
field_index = []
X_modified = pd.DataFrame()

for index, col in enumerate(X_train.columns):
    
    # if col in DUMMY_FIELDS:
    #     type_id = pd.get_dummies(X_train[col], prefix=col, prefix_sep='-')
    #     field_dict[index] = list(type_id.columns)
    #     field_index.extend(repeat(index, type_id.shape[1]))
    #     X_modified = pd.concat([X_modified, type_id], axis=1)

    if col in CAT_FIELDS:
        one_hot = MultiLabelBinarizer()
        X_cat_col = pd.DataFrame(one_hot.fit_transform(X_train[col].fillna('None')), columns=one_hot.classes_)
        field_dict[index] = list(X_cat_col.columns)
        field_index.extend(repeat(index, X_cat_col.shape[1]))
        X_modified = pd.concat([X_modified, X_cat_col], axis=1)

    if col in CONT_FIELDS:
        scaler = MinMaxScaler()
        X_cont_col = pd.DataFrame(scaler.fit_transform(X_train[[col]]), columns=[col])
        field_dict[index] = col
        field_index.append(index)
        X_modified = pd.concat([X_modified, X_cont_col], axis=1)


X_train_modified = X_modified

In [ ]:
# X_test field에 맞춰 modify
field_dict = dict()
field_index = []
X_modified = pd.DataFrame()

for index, col in enumerate(X_test.columns):

    # if col in DUMMY_FIELDS:
    #     type_id = pd.get_dummies(X_test[col], prefix=col, prefix_sep='-')
    #     field_dict[index] = list(type_id.columns)
    #     field_index.extend(repeat(index, type_id.shape[1]))
    #     X_modified = pd.concat([X_modified, type_id], axis=1)

    if col in CAT_FIELDS:
        one_hot = MultiLabelBinarizer()
        X_cat_col = pd.DataFrame(one_hot.fit_transform(X_test[col].fillna('None')), columns=one_hot.classes_)
        field_dict[index] = list(X_cat_col.columns)
        field_index.extend(repeat(index, X_cat_col.shape[1]))
        X_modified = pd.concat([X_modified, X_cat_col], axis=1)

    if col in CONT_FIELDS:
        scaler = MinMaxScaler()
        X_cont_col = pd.DataFrame(scaler.fit_transform(X_test[[col]]), columns=[col])
        field_dict[index] = col
        field_index.append(index)
        X_modified = pd.concat([X_modified, X_cont_col], axis=1)


X_test_modified = X_modified

In [ ]:
# 결측 mean으로 fillna
for col in X_train_modified:
    if X_train_modified[col].isnull().sum():
        X_train_modified[col].fillna(X_train_modified[col].mean(), inplace=True)

for col in X_test_modified:
    if X_test_modified[col].isnull().sum():
        X_test_modified[col].fillna(X_test_modified[col].mean(), inplace=True)

class 구현

In [ ]:
class FM_layer(tf.keras.layers.Layer):
    def __init__(self, num_feature, num_field, embedding_size, field_index):
        super(FM_layer, self).__init__()
        self.embedding_size = embedding_size    # k: 임베딩 벡터의 차원(크기)
        self.num_feature = num_feature          # f: 원래 feature 개수
        self.num_field = num_field              # m: grouped field 개수
        self.field_index = field_index          # 인코딩된 X의 칼럼들이 본래 어디 소속이었는지

        # Parameters of FM Layer
        # w: capture 1st order interactions
        # V: capture 2nd order interactions
        self.w = tf.Variable(tf.random.normal(shape=[num_feature],
                                              mean=0.0, stddev=1.0), name='w')
        self.V = tf.Variable(tf.random.normal(shape=(num_field, embedding_size),
                                              mean=0.0, stddev=0.01), name='V')

    def call(self, inputs):
        x_batch = tf.reshape(inputs, [-1, self.num_feature, 1])  # inputs:(256, 38), x_batch:(256, 38, 1)
        # Parameter V를 field_index에 맞게 복사하여 num_feature에 맞게 늘림
        embeds = tf.nn.embedding_lookup(params=self.V, ids=self.field_index)  # V:(14, 5), embeds:(38, 5)

        # Deep Component에서 쓸 Input
        # (batch_size, num_feature, embedding_size)
        new_inputs = tf.math.multiply(x_batch, embeds) # (256, 38, 5)

        # (batch_size, )
        linear_terms = tf.reduce_sum(
            tf.math.multiply(self.w, inputs), axis=1, keepdims=False)

        # (batch_size, )
        interactions = 0.5 * tf.subtract(
            tf.square(tf.reduce_sum(new_inputs, [1, 2])),
            tf.reduce_sum(tf.square(new_inputs), [1, 2])
        )

        linear_terms = tf.reshape(linear_terms, [-1, 1])
        interactions = tf.reshape(interactions, [-1, 1])

        y_fm = tf.concat([linear_terms, interactions], 1) # (256, 2)

        return y_fm, new_inputs


In [ ]:
class DeepFM(tf.keras.Model):

    def __init__(self, num_feature, num_field, embedding_size, field_index):
        super(DeepFM, self).__init__()
        self.embedding_size = embedding_size    # k: 임베딩 벡터의 차원(크기)
        self.num_feature = num_feature          # f: 원래 feature 개수
        self.num_field = num_field              # m: grouped field 개수
        self.field_index = field_index          # 인코딩된 X의 칼럼들이 본래 어디 소속이었는지

        self.fm_layer = FM_layer(num_feature, num_field, embedding_size, field_index)

        self.layers1 = tf.keras.layers.Dense(units=64, activation='relu')
        self.dropout1 = tf.keras.layers.Dropout(rate=0.2)
        self.layers2 = tf.keras.layers.Dense(units=16, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(rate=0.2)
        self.layers3 = tf.keras.layers.Dense(units=2, activation='relu')

        self.final = tf.keras.layers.Dense(units=1, activation='sigmoid')

    def __repr__(self):
        return "DeepFM Model: #Field: {}, #Feature: {}, ES: {}".format(
            self.num_field, self.num_feature, self.embedding_size)

    def call(self, inputs):
        # 1) FM Component: (num_batch, 2)
        y_fm, new_inputs = self.fm_layer(inputs)

        # retrieve Dense Vectors: (num_batch, num_feature*embedding_size)
        new_inputs = tf.reshape(new_inputs, [-1, self.num_feature*self.embedding_size])

        # 2) Deep Component
        y_deep = self.layers1(new_inputs)
        y_deep = self.dropout1(y_deep)
        y_deep = self.layers2(y_deep)
        y_deep = self.dropout2(y_deep)
        y_deep = self.layers3(y_deep)

        # Concatenation
        y_pred = tf.concat([y_fm, y_deep], 1)
        y_pred = self.final(y_pred)
        y_pred = tf.reshape(y_pred, [-1, ])

        return y_pred


In [ ]:
# Batch 나눈 dataset  --> BATCH_SIZE 변경할 경우 해당 셀 다시 돌려줘야 함
train_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_train_modified.values, tf.float32), tf.cast(y_train, tf.float32))) \
    .shuffle(30000).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_test_modified.values, tf.float32), tf.cast(y_test, tf.float32))) \
    .shuffle(10000).batch(BATCH_SIZE)


features : item meta (user meta X) & type_id (continuous) + food (categorical)

In [ ]:
# Batch 단위 학습
def train_on_batch(model, optimizer,inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        # y_pred = tf.where(tf.math.is_nan(y_pred), tf.zeros_like(y_pred), y_pred)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False, y_true=targets, y_pred=y_pred)

    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용함
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return model


# 반복 학습 함수
def train(epochs):
    model = DeepFM(embedding_size=EMBEDDING_SIZE, num_feature=len(field_index),
                   num_field=len(field_dict), field_index=field_index)

    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    print("Start Training: Batch Size: {}, Embedding Size: {}".format(BATCH_SIZE, EMBEDDING_SIZE))
    
    for i in range(epochs):
        for x, y in train_ds:
            model = train_on_batch(model, optimizer, x, y)

    return model


# if __name__ == '__main__':
#     train(epochs=EPOCHS)


# sklearn metrics
start = perf_counter()
model = train(EPOCHS)
y_pred = model(X_test_modified.values).numpy().round()
# y_pred = tf.where(tf.math.is_nan(y_pred), tf.zeros_like(y_pred), y_pred)
test_acc = accuracy_score(y_test.values, y_pred)
test_auc = roc_auc_score(y_test.values, y_pred)
test_prec = precision_score(y_test.values, y_pred)

print("테스트 ACC: {:.4f}, AUC: {:.4f}, PREC: {:.4f}".format(test_acc, test_auc, test_prec))
print("걸린 시간: {:.3f}".format(perf_counter() - start))
# model.save_weights('weights/weights-epoch({})-batch({})-embedding({}).h5'.format(
    # epochs, BATCH_SIZE, EMBEDDING_SIZE))

Start Training: Batch Size: 256, Embedding Size: 5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

테스트 ACC: 0.6523, AUC: 0.6571, PREC: 0.7112
걸린 시간: 648.624


###input user id에게 와인 추천

모델 학습

In [ ]:
# Batch 단위 학습
def train_on_batch(model, optimizer,inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        # y_pred = tf.where(tf.math.is_nan(y_pred), tf.zeros_like(y_pred), y_pred)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False, y_true=targets, y_pred=y_pred)

    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용함
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return model


# 반복 학습 함수
def train(epochs):
    model = DeepFM(embedding_size=EMBEDDING_SIZE, num_feature=len(field_index),
                   num_field=len(field_dict), field_index=field_index)

    optimizer = tf.keras.optimizers.Adam()

    print("Start Training: Batch Size: {}, Embedding Size: {}".format(BATCH_SIZE, EMBEDDING_SIZE))
    
    for i in range(epochs):
        for x, y in train_ds:
            model = train_on_batch(model, optimizer, x, y)

    return model

features : user & item meta (continuous) + food (categorical)

In [ ]:
# sklearn metrics
model = train(EPOCHS)
y_pred = model(tf.cast(X_test_modified.values, tf.float64)).numpy().round()
# y_pred = tf.where(tf.math.is_nan(y_pred), tf.zeros_like(y_pred), y_pred)
test_acc = accuracy_score(y_test.values, y_pred)
test_auc = roc_auc_score(y_test.values, y_pred)
test_prec = precision_score(y_test.values, y_pred)

print("테스트 ACC: {:.4f}, AUC: {:.4f}, PREC: {:.4f}".format(test_acc, test_auc, test_prec))

Start Training: Batch Size: 256, Embedding Size: 1
테스트 ACC: 0.6575, AUC: 0.6590, PREC: 0.6947


input user에게 top K개 와인 추천

In [ ]:
all_user_meta = pd.concat([train_df.userID, X_train_modified[X_train_modified.columns[-5:]]], axis=1).drop_duplicates()
all_item_meta = pd.concat([train_df.wine_id, X_train_modified[X_train_modified.columns[:-5]]], axis=1).drop_duplicates()

In [ ]:
input_user = 1201  # 추천해줄 유저 id
input_user_meta = all_user_meta.loc[all_user_meta.userID == input_user,:].values[0][1:]
input_all = pd.DataFrame(columns=X_train_modified.columns)
for item in all_item_meta.values:
    tmp = np.concatenate((item[1:], input_user_meta))
    input_all = input_all.append(pd.Series(tmp, index=X_train_modified.columns, name=item[0]))

In [ ]:
y_pred = model(input_all.values).numpy()
y_pred = pd.Series(y_pred, name='y_pred')
wine_id = all_item_meta.wine_id.reset_index(drop=True)
input_predict = pd.concat([wine_id, y_pred], axis=1)
input_predict = input_predict.sort_values(by='y_pred', ascending=False)

K = 50
predict_top_k = input_predict.wine_id[:K].values
predict_top_k

array([3799668,   77178, 1307996, 2634814, 1654616, 3925126, 1639135,
       1591248, 1140747, 1235861, 1637757, 5281856, 4151209, 2130162,
       1598998, 2395864,   77137, 5583054,   75975, 4154371, 8075591,
       6005900, 2545450, 1220504, 2556060, 3851590, 6414288, 5420759,
         82034, 2344981, 1293594, 7088380, 1271002, 4110288, 1505859,
       1144145, 1180996, 1562648, 4382344, 1990760, 1248773, 3959683,
       1403662, 2104668, 2270223, 5154081, 2495612,   43924, 5354893,
       1186732])

In [ ]:
interact_wine_id = train_df.loc[train_df.userID==input_user,['wine_id','like']].wine_id.values
like_wine_id = train_df.loc[(train_df.userID==input_user) & (train_df.like==1),['wine_id','like']].wine_id.values  # 유저가 실제로 좋아한 wine id
like_wine_id

array([1845754,   13563, 1236122, 1915266, 1101359,   15162,  100791,
       2003036, 1200716, 1127795, 1684223, 1224338,   66104, 1146850,
       1187200,   10922,  768733, 1138019, 2714924, 1134803, 1605285,
        100046, 1374665,   83444, 4383995, 2273709, 3506976,   83913,
       1172428, 1419005, 1098851,   36434, 1100673,   88387,  100390,
         79235, 1563404, 1100147,    9240, 1193557, 1115658, 1125543,
       1167825, 1098887, 1513469,  100044, 1885312,   84899, 1146850,
       1099253, 1152755, 2078055,   82199,   12008,   79233,   88177,
         79231, 1214948,  100047,   91173,   81848,   83912,   77136,
       1840114,   79926, 1655970,    3908, 1127795,   75846, 1257923,
         79635,    3908, 1655970, 2242419,   83920,   83911, 1100910,
         85063,   77137, 1167289,   85053, 1172428, 2343086, 2103498,
       2103498])

In [ ]:
set(predict_top_k).intersection(like_wine_id)  # 추천해준 wine_id 중 실제 유저가 좋아한 와인

{77137}

In [ ]:
all_user_meta = pd.concat([train_df.userID, X_train_modified[X_train_modified.columns[-5:]]], axis=1).drop_duplicates().reset_index(drop=True)
all_item_meta = pd.concat([train_df.wine_id, X_train_modified[X_train_modified.columns[:-5]]], axis=1).drop_duplicates().reset_index(drop=True)

In [ ]:
# 전체 user id 예측 -> 실제 좋아한 와인 예측 건수
wine_id = all_item_meta.wine_id.reset_index(drop=True)
K = 50
recommend_top_list = []
true_recommend = []

for input_user in all_user_meta.userID:
    
    input_user_meta = pd.DataFrame(itertools.repeat(all_user_meta.loc[all_user_meta.userID == input_user,:].values[0][1:], len(all_item_meta)), columns=X_train_modified.columns[-5:])
    input_all = pd.concat([all_item_meta.iloc[:,1:], input_user_meta], axis=1)

    y_pred = model(input_all.values).numpy()
    y_pred = pd.Series(y_pred, name='y_pred')
    input_predict = pd.concat([wine_id, y_pred], axis=1)
    input_predict = input_predict.sort_values(by='y_pred', ascending=False)
    predict_top_k = input_predict.wine_id[:K].values

    recommend_top_list.append(predict_top_k)
    like_wine_id = train_df.loc[(train_df.userID==input_user) & (train_df.like==1),'wine_id'].values
    true_recommend.append(len(set(predict_top_k).intersection(like_wine_id)))

pd.Series(true_recommend).value_counts()

0    5500
1     643
2     142
3      49
5       5
4       3
7       1
dtype: int64

대부분 유저가 실제로 좋아한 와인은 추천해주지 않는다.